<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/04_hsv-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-16 23:08:31--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11745 (11K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]  11.47K  --.-KB/s    in 0s      

2022-06-16 23:08:32 (114 MB/s) - ‘utils.py.1’ saved [11745/11745]



In [2]:
from google.colab import drive
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub


from utils import ModelCreator, get_train_val_ds, get_class_weight

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
hsv_creator = ModelCreator(HUB_URL, 'HSV-model')
hsv_model = hsv_creator.make_model(img_size=IMG_SIZE)

Model: "HSV-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
Total params: 23,533,136
Trainable params: 32,784
Non-trainable params: 23,500,352
_________________________________________________________________


In [10]:
callbacks = hsv_creator.get_callbacks()

In [11]:
history = hsv_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=20,
                        class_weight=class_weight)

Epoch 1/20
519/519 [==============================] - 299s 542ms/step - loss: 155.6929 - auc: 0.2850 - accuracy: 0.3896
Epoch 2/20
519/519 [==============================] - 148s 283ms/step - loss: 107.0318 - auc: 0.4021 - accuracy: 0.5098
Epoch 3/20
519/519 [==============================] - 148s 282ms/step - loss: 89.9372 - auc: 0.4712 - accuracy: 0.5679
Epoch 4/20
519/519 [==============================] - 148s 282ms/step - loss: 75.6156 - auc: 0.5188 - accuracy: 0.6052
Epoch 5/20
519/519 [==============================] - 148s 282ms/step - loss: 61.1100 - auc: 0.5729 - accuracy: 0.6478
Epoch 6/20
519/519 [==============================] - 148s 283ms/step - loss: 58.5743 - auc: 0.5844 - accuracy: 0.6554
Epoch 7/20
519/519 [==============================] - 148s 282ms/step - loss: 45.6877 - auc: 0.6340 - accuracy: 0.6866
Epoch 8/20
519/519 [==============================] - 148s 282ms/step - loss: 37.6064 - auc: 0.6643 - accuracy: 0.7055
Epoch 9/20
519/519 [==========================

In [17]:
!cp ./logs/checkpoints/HSV-model-10.h5 ./drive/MyDrive/checkpoints/hsv-model-10.h5
!cp ./logs/checkpoints/HSV-model-14.h5 ./drive/MyDrive/checkpoints/hsv-model-14.h5
!cp ./logs/checkpoints/HSV-model-20.h5 ./drive/MyDrive/checkpoints/hsv-model-20.h5

In [13]:
history = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=30,
                          class_weight=class_weight,
                          initial_epoch=20)

Epoch 21/30
519/519 [==============================] - 147s 280ms/step - loss: 6.0910 - auc: 0.9097 - accuracy: 0.8710
Epoch 22/30
519/519 [==============================] - 148s 282ms/step - loss: 5.6916 - auc: 0.9191 - accuracy: 0.8753
Epoch 23/30
519/519 [==============================] - 147s 282ms/step - loss: 5.0233 - auc: 0.9294 - accuracy: 0.8855
Epoch 24/30
519/519 [==============================] - 148s 282ms/step - loss: 4.5864 - auc: 0.9343 - accuracy: 0.8899
Epoch 25/30
519/519 [==============================] - 148s 282ms/step - loss: 4.1191 - auc: 0.9423 - accuracy: 0.8963
Epoch 26/30
519/519 [==============================] - 148s 283ms/step - loss: 3.5427 - auc: 0.9502 - accuracy: 0.9050
Epoch 27/30
519/519 [==============================] - 147s 281ms/step - loss: 3.5354 - auc: 0.9488 - accuracy: 0.9052
Epoch 28/30
519/519 [==============================] - 147s 282ms/step - loss: 2.9942 - auc: 0.9593 - accuracy: 0.9166
Epoch 29/30
519/519 [===========================

In [15]:
!cp ./logs/checkpoints/HSV-model-25.h5 ./drive/MyDrive/checkpoints/hsv-model-25.h5
!cp ./logs/checkpoints/HSV-model-30.h5 ./drive/MyDrive/checkpoints/hsv-model-30.h5

In [16]:
history = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=40,
                          class_weight=class_weight,
                          initial_epoch=30)

Epoch 31/40
519/519 [==============================] - 148s 283ms/step - loss: 2.4778 - auc: 0.9648 - accuracy: 0.9261
Epoch 32/40
519/519 [==============================] - 148s 282ms/step - loss: 2.2595 - auc: 0.9693 - accuracy: 0.9295
Epoch 33/40
519/519 [==============================] - 149s 284ms/step - loss: 2.1963 - auc: 0.9699 - accuracy: 0.9339
Epoch 34/40
519/519 [==============================] - 148s 283ms/step - loss: 2.0398 - auc: 0.9713 - accuracy: 0.9357
Epoch 35/40
519/519 [==============================] - 150s 286ms/step - loss: 1.9213 - auc: 0.9732 - accuracy: 0.9411
Epoch 36/40
519/519 [==============================] - 148s 282ms/step - loss: 1.8671 - auc: 0.9736 - accuracy: 0.9407
Epoch 37/40
519/519 [==============================] - 148s 283ms/step - loss: 1.7814 - auc: 0.9752 - accuracy: 0.9448
Epoch 38/40
519/519 [==============================] - 148s 282ms/step - loss: 1.7170 - auc: 0.9758 - accuracy: 0.9476
Epoch 39/40
519/519 [===========================

In [18]:
!cp ./logs/checkpoints/HSV-model-35.h5 ./drive/MyDrive/checkpoints/hsv-model-35.h5
!cp ./logs/checkpoints/HSV-model-39.h5 ./drive/MyDrive/checkpoints/hsv-model-39.h5

In [19]:
history = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=50,
                          class_weight=class_weight,
                          initial_epoch=40)

Epoch 41/50
519/519 [==============================] - 149s 285ms/step - loss: 1.5272 - auc: 0.9785 - accuracy: 0.9508
Epoch 42/50
519/519 [==============================] - 149s 284ms/step - loss: 1.4949 - auc: 0.9789 - accuracy: 0.9507
Epoch 43/50
519/519 [==============================] - 147s 282ms/step - loss: 1.4616 - auc: 0.9796 - accuracy: 0.9524
Epoch 44/50
519/519 [==============================] - 148s 282ms/step - loss: 1.4314 - auc: 0.9800 - accuracy: 0.9541
Epoch 45/50
519/519 [==============================] - 148s 282ms/step - loss: 1.3965 - auc: 0.9803 - accuracy: 0.9543
Epoch 46/50
519/519 [==============================] - 147s 282ms/step - loss: 1.3419 - auc: 0.9817 - accuracy: 0.9562
Epoch 47/50
519/519 [==============================] - 148s 282ms/step - loss: 1.3279 - auc: 0.9819 - accuracy: 0.9557
Epoch 48/50
519/519 [==============================] - 148s 283ms/step - loss: 1.3176 - auc: 0.9817 - accuracy: 0.9563
Epoch 49/50
519/519 [===========================

In [21]:
!cp ./logs/checkpoints/HSV-model-45.h5 ./drive/MyDrive/checkpoints/hsv-model-45.h5
!cp ./logs/checkpoints/HSV-model-50.h5 ./drive/MyDrive/checkpoints/hsv-model-50.h5